In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, r"/home/rupeshknn/projects/qc/pulse-backend-qamp22/qiskit-experiments/test/")

In [3]:
import unittest
from qiskit_experiments.test.iq_pulse_backend import SingleTransmonTestBackend
pulse_backend = SingleTransmonTestBackend(noise=False)

In [4]:
"""Test rough amplitude calibration experiment classes."""

from base import QiskitExperimentsTestCase
import numpy as np

from qiskit import transpile
import qiskit.pulse as pulse
from qiskit.circuit import Parameter
from qiskit.providers.fake_provider import FakeArmonkV2

from qiskit_experiments.calibration_management.basis_gate_library import FixedFrequencyTransmon
from qiskit_experiments.calibration_management import Calibrations
from qiskit_experiments.library import EFRoughXSXAmplitudeCal, RoughXSXAmplitudeCal
from qiskit_experiments.test.mock_iq_backend import MockIQBackend
from qiskit_experiments.test.mock_iq_helpers import MockIQRabiHelper as RabiHelper

In [5]:
class TestRoughAmpCal(QiskitExperimentsTestCase):
    """A class to test the rough amplitude calibration experiments."""

    def setUp(self):
        """Setup the tests."""
        super().setUp()
        library = FixedFrequencyTransmon()

        self.backend = SingleTransmonTestBackend(noise=False) #FakeArmonkV2()
        self.cals = Calibrations.from_backend(self.backend, libraries=[library])

    def test_circuits(self):
        """Test the quantum circuits."""
        test_amps = [-0.5, 0, 0.5]
        rabi = RoughXSXAmplitudeCal(0, self.cals, amplitudes=test_amps)

        circs = transpile(rabi.circuits(), self.backend, inst_map=self.cals.default_inst_map)

        for circ, amp in zip(circs, test_amps):
            self.assertEqual(circ.count_ops()["Rabi"], 1)

            d0 = pulse.DriveChannel(0)
            with pulse.build(name="x") as expected_x:
                pulse.play(pulse.Drag(160, amp, 40, 0), d0)

            self.assertEqual(circ.calibrations["Rabi"][((0,), (amp,))], expected_x)

    def test_update(self):
        """Test that the calibrations update properly."""
        tol = 0.01
        default_amp = 0.5 / self.backend.rabi_rate_01
        
        rabi_ef = RoughXSXAmplitudeCal(0, self.cals, amplitudes=np.linspace(-0.1, 0.1, 21))
        expdata = rabi_ef.run(self.backend)
        self.assertExperimentDone(expdata)
                
        self.assertAlmostEqual(self.cals.get_parameter_value("amp", 0, "x"), default_amp, delta=tol)
        self.assertAlmostEqual(self.cals.get_parameter_value("amp", 0, "sx"), default_amp/2, delta=tol)
        
        self.cals.add_parameter_value(int(4*160/5),'duration',(),schedule='x')
        rabi_ef = RoughXSXAmplitudeCal(0, self.cals, amplitudes=np.linspace(-0.1, 0.1, 21))
        expdata = rabi_ef.run(self.backend)
        self.assertExperimentDone(expdata)
        self.rabi = rabi_ef
        
        self.assertTrue(abs(self.cals.get_parameter_value("amp", 0, "x")*(4/5) - default_amp) < tol)
        self.assertTrue(abs(self.cals.get_parameter_value("amp", 0, "sx")*(4/5) - default_amp / 2) < tol)

    def test_experiment_config(self):
        """Test converting to and from config works"""
        exp = RoughXSXAmplitudeCal(0, self.cals)
        config = exp.config()
        loaded_exp = RoughXSXAmplitudeCal.from_config(config)
        self.assertNotEqual(exp, loaded_exp)
        self.assertEqual(config, loaded_exp.config())

In [6]:
# obj = TestRoughAmpCal("test_update")
# obj.setUp()
# obj.test_update()

suite = unittest.makeSuite(TestRoughAmpCal, 'test')
TestRoughAmpCal_result = unittest.TestResult()
suite.run(TestRoughAmpCal_result)

/home/rupeshknn/mambaforge/envs/exp/lib/python3.9/site-packages/numpy/lib/function_base.py:2411: RuntimeWarning: invalid value encountered in <lambda> (vectorized)
  outputs = ufunc(*inputs)


<unittest.result.TestResult run=3 errors=0 failures=0>

In [7]:
class TestSpecializations(QiskitExperimentsTestCase):
    """Test the specialized versions of the calibration."""

    def setUp(self):
        """Setup the tests"""
        super().setUp()

        library = FixedFrequencyTransmon()

        self.backend = SingleTransmonTestBackend(noise=False)
        self.cals = Calibrations.from_backend(self.backend, libraries=[library])

        # Add some pulses on the 1-2 transition.
        d0 = pulse.DriveChannel(0)
        with pulse.build(name="x12") as x12:
            with pulse.frequency_offset(-300e6, d0):
                pulse.play(pulse.Drag(Parameter("duration"), Parameter("amp"), 40, 0.0), d0)

        with pulse.build(name="sx12") as sx12:
            with pulse.frequency_offset(-300e6, d0):
                pulse.play(pulse.Drag(Parameter("duration"), Parameter("amp"), 40, 0.0), d0)

        self.cals.add_schedule(x12, 0)
        self.cals.add_schedule(sx12, 0)
        self.cals.add_parameter_value(0.4, "amp", 0, "x12")
        self.cals.add_parameter_value(0.2, "amp", 0, "sx12")
        self.cals.add_parameter_value(160, "duration", 0, "x12")
        self.cals.add_parameter_value(160, "duration", 0, "sx12")

    def test_ef_circuits(self):
        """Test that we get the expected circuits with calibrations for the EF experiment."""

        test_amps = [-0.5, 0, 0.5]
        rabi_ef = EFRoughXSXAmplitudeCal(0, self.cals, amplitudes=test_amps)

        circs = transpile(rabi_ef.circuits(), self.backend, inst_map=self.cals.default_inst_map)

        for circ, amp in zip(circs, test_amps):

            self.assertEqual(circ.count_ops()["x"], 1)
            self.assertEqual(circ.count_ops()["Rabi"], 1)

            d0 = pulse.DriveChannel(0)
            with pulse.build(name="x") as expected_x:
                pulse.play(pulse.Drag(160, 0.5, 40, 0), d0)

            with pulse.build(name="x12") as expected_x12:
                with pulse.frequency_offset(-300e6, d0):
                    pulse.play(pulse.Drag(160, amp, 40, 0), d0)

            self.assertEqual(circ.calibrations["x"][((0,), ())], expected_x)
            self.assertEqual(circ.calibrations["Rabi"][((0,), (amp,))], expected_x12)

    def test_ef_update(self):
        """Tes that we properly update the pulses on the 1<->2 transition."""
        tol = 0.01
        default_amp = 0.5 / self.backend.rabi_rate_12
        
        rabi_ef = EFRoughXSXAmplitudeCal(0, self.cals, amplitudes=np.linspace(-0.1, 0.1, 21))
        expdata = rabi_ef.run(self.backend)
        self.assertExperimentDone(expdata)
        
        self.assertAlmostEqual(self.cals.get_parameter_value("amp", 0, "x12"), default_amp, delta=tol)
        self.assertAlmostEqual(self.cals.get_parameter_value("amp", 0, "sx12"), default_amp/2, delta=tol)
        
        
        self.cals.add_parameter_value(int(4*160/5), "duration", 0, "x12")
        self.cals.add_parameter_value(int(4*160/5), "duration", 0, "sx12")
        rabi_ef2 = EFRoughXSXAmplitudeCal(0, self.cals, amplitudes=np.linspace(-0.1, 0.1, 31))
        self.rabi = rabi_ef2
        expdata = rabi_ef2.run(self.backend)
        self.assertExperimentDone(expdata)
        

        self.assertTrue(abs(self.cals.get_parameter_value("amp", 0, "x12")*(4/5) - default_amp) < tol)
        self.assertTrue(abs(self.cals.get_parameter_value("amp", 0, "sx12")*(4/5) - default_amp / 2) < tol)

In [8]:
# obj = TestSpecializations("test_ef_update")
# obj.setUp()
# obj.test_ef_update()

suite = unittest.makeSuite(TestSpecializations, 'test')
TestSpecializations_result = unittest.TestResult()
suite.run(TestSpecializations_result)

/home/rupeshknn/mambaforge/envs/exp/lib/python3.9/site-packages/numpy/lib/function_base.py:2411: RuntimeWarning: invalid value encountered in <lambda> (vectorized)
  outputs = ufunc(*inputs)


<unittest.result.TestResult run=2 errors=0 failures=0>